### ライブラリのインポート

In [1]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd

### HTMLから欲しいテキストを抽出する

In [2]:
urls = [
    "https://magi.camp/blogs/167305889",
    "https://magi.camp/blogs/47058427",  
    "https://magi.camp/blogs/774748184" 
]

# 取得したテキストをまとめるリスト
html_texts = []

for url in urls:
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    lines = soup.get_text(separator="\n").split("\n")
    html_texts.append(lines)

### 抽出したテキストから、「発売日、希望小売価格、カード枚数」を切り分ける

In [3]:
data = []

for lines in html_texts:
    i = 0
    while i < len(lines):
        line = lines[i].strip()

        if "発売日" in line:
            release = re.search(r"発売日[:：]?\s*(\d{4}年\d{1,2}月\d{1,2}日)", line)
            price   = re.search(r"希望小売価格[:：]?\s*([0-9,]+円)", lines[i+1].strip()) if i+1 < len(lines) else None
            count   = re.search(r"カード[／/]\s*([0-9]+)枚", lines[i+2].strip()) if i+2 < len(lines) else None

            if release and price and count:
                data.append({
                    "発売日": release.group(1),
                    "希望小売価格": price.group(1),
                    "カード枚数": count.group(1)
                })
                i += 3
            else:
                i += 1
        else:
            i += 1

### 切り分けたデータをデータフレームにまとめて、CSVで出力する

In [5]:
df = pd.DataFrame(data)
df.to_csv("pokemon_cards_all_sites.csv", index=False, encoding="utf-8-sig")